# Symmetrie
Um das Caching noch effektiver zu gestalten, sollen neben Transpositionen auch Symmetrien erkannt werden. In diesem Kapitel werden alle Funktionen, die für die Symmetrieerkennung nötig sind, vorgestellt und implementiert.

Zunächst werden Hilfsfunktion definiert, die auf den gegebenen Spielbrettern (`boards`) eine bestimmte Symmetrie anwenden und alle resultierenden Spielbretter in einer Menge (`Set`) zurück geben. Schlussendlich werden alle Symmetrien nacheinander angewandt, damit auch zusammengesetzte Symetrien wie beispielsweise `Rotation um 90°` dann `Spiegelung an der horizontalen Achse` errechnet werden.

In [ ]:
from IPython.core.display import HTML
with open("style.html", "r") as file:
    css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game-utils.ipynb

## Rotation
Ein Spielbrett kann um 90°, 180° oder 270° gedreht werden, die resultierenden Spielbretter sind rotationssymmetrisch.

Die Eingabe besteht aus einer Menge von Spielbrettern (`boards`), die Ausgabe ist ebenfalls eine Menge, die alle Spielbretter enthält, die rotationssymmetrisch zu der Eingabe sind.
Berechnet wird die Ausgabe indem alle Ringe um $k \in {2, 4, 6}$ Zellen rotiert werden. Durch Aneinanderreihung der letzten $8-k$ Zellen und der ersten $k$ Zellen kommt die Rotation zustande.

In [ ]:
def symmetryRotation(boards):
    return {
        tuple(
            board[ring][rotation:] + board[ring][:rotation]
            for ring in range(3)
        )
        for rotation in range(2, 6+1, 2)
        for board in boards
    }

## Spiegelung
Bei den Spiegelungen wird an vier Achsen gespiegelt:

* die *horizontale* und *vertikale* Achse, sowie
* die Diagonale von oben links nach unten rechts (*negative Diagonale*) und die Diagonale von unten links nach oben rechts (*positive Diagonale*).

Diese Spiegelungen können einzelnd pro Ring vorgenommen werden, da der äußere Ring bleibt nach der Spiegelung weiterhin der äußere Ring. Gleiches gilt für die anderen Ringe. Alle Spiegelungen lassen sich durch eine Invertierung der Ringe und eine Rotation von $k \in {0, 2, 4, 6}$ darstellen.

In [ ]:
def symmetryHorizontal(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-(cell+2))%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

In [ ]:
def symmetryVertical(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-(cell+6))%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

In [ ]:
def symmetryDiagonalPositive(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-(cell+4))%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

In [ ]:
def symmetryDiagonalNegative(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-cell)%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

## Ring-Tausch
Da der innere und der äußere Ring über symmetrische Kanten mit dem mittleren Ring verbunden ist, können der äußere und der innere Ringe getauscht werden. Dies funktioniert indem rückwärts über die Ringe iteriert wird.

In [ ]:
def symmetryRing(boards):
    return {
        tuple(
            board[ring]
            for ring in reversed(range(3))
        )
        for board in boards
    }

## Zusammenführung
Damit alle möglichen Symmetrien gefunden werden, wird jede Hilfsfunktion einzelnd auf alle vorherigen Spielbretter (`boards`) oder Zustände (`states`) angewandt. Dadurch sind auch zusammengesetzte Symmetrien wie beispielsweise `Rotation um 90°` dann `Spiegelung an der horizontalen Achse` möglich. Mit Hilfe einer Menge wird sichergestellt, dass keine Duplikate zurück gegeben werden.

In [ ]:
def findSymmetries(state):
    stash, board = state
    
    boards = { board }
    boards |= symmetryRotation(boards)
    boards |= symmetryHorizontal(boards)
    boards |= symmetryVertical(boards)
    boards |= symmetryDiagonalPositive(boards)
    boards |= symmetryDiagonalNegative(boards)
    boards |= symmetryRing(boards)
    
    return {
        (stash, board)
        for board in boards
    }